In [7]:
# This is Main function.
# Extracting streaming data from Twitter, pre-processing, and loading into MySQL
import credentials # Import api/access_token keys from credentials.py
import settings # Import related setting constants from settings.py 

import re
import tweepy
import mysql.connector
import pandas as pd
from textblob import TextBlob
# Streaming With Tweepy 
# http://docs.tweepy.org/en/v3.4.0/streaming_how_to.html#streaming-with-tweepy

In [8]:
# Override tweepy.StreamListener to add logic to on_status
class MyStreamListener(tweepy.StreamListener):
    '''
    Tweets are known as “status updates”. So the Status class in tweepy has properties describing the tweet.
    https://developer.twitter.com/en/docs/tweets/data-dictionary/overview/tweet-object.html
    '''
    
    def on_status(self, status):
        '''
        Extract info from tweets
        '''
        
        if status.retweeted:
            # Avoid retweeted info, and only original tweets will be received
            return True
        # Extract attributes from each tweet
        id_str = status.id_str
        created_at = status.created_at
        text = deEmojify(status.text)    # Pre-processing the text  
        sentiment = TextBlob(text).sentiment
        polarity = sentiment.polarity
        subjectivity = sentiment.subjectivity
        
        user_created_at = status.user.created_at
        user_location = deEmojify(status.user.location)
        user_description = deEmojify(status.user.description)
        user_followers_count =status.user.followers_count
        longitude = None
        latitude = None
        if status.coordinates:
            longitude = status.coordinates['coordinates'][0]
            latitude = status.coordinates['coordinates'][1]
            
        retweet_count = status.retweet_count
        favorite_count = status.favorite_count
        
        print(status.text)
        print("Long: {}, Lati: {}".format(longitude, latitude))
        
        # Store all data in MySQL
        if mydb.is_connected():
            mycursor = mydb.cursor()
            sql = "INSERT INTO {} (id_str, created_at, text, polarity, subjectivity, user_created_at, user_location, user_description, user_followers_count, longitude, latitude, retweet_count, favorite_count) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)".format(settings.TABLE_NAME)
            val = (id_str, created_at, text, polarity, subjectivity, user_created_at, user_location, \
                user_description, user_followers_count, longitude, latitude, retweet_count, favorite_count)
            mycursor.execute(sql, val)
            mydb.commit()
            mycursor.close()
    
    
    def on_error(self, status_code):
        '''
        Since Twitter API has rate limits, stop srcraping data as it exceed to the thresold.
        '''
        if status_code == 420:
            # return False to disconnect the stream
            return False

In [9]:
def clean_tweet(self, tweet): 
    ''' 
    Use sumple regex statemnents to clean tweet text by removing links and special characters
    '''
    return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t]) \
                                |(\w+:\/\/\S+)", " ", tweet).split()) 
def deEmojify(text):
    '''
    Strip all non-ASCII characters to remove emoji characters
    '''
    if text:
        return text.encode('ascii', 'ignore').decode('ascii')
    else:
        return None

In [10]:
mydb = mysql.connector.connect(
    host="localhost",
    user="root",
    passwd="Datasci92!",
    database="Twitter",
    charset = 'utf8'
)#prbly have to create table via mysql
if mydb.is_connected():
    '''
    Check if this table exits. If not, then create a new one.
    '''
    mycursor = mydb.cursor()
    mycursor.execute("""
        SELECT COUNT(*)
        FROM information_schema.tables
        WHERE table_name = '{0}'
        """.format(settings.TABLE_NAME))
    if mycursor.fetchone()[0] != 1:
        mycursor.execute("CREATE TABLE {} ({})".format(settings.TABLE_NAME, settings.TABLE_ATTRIBUTES))
        mydb.commit()
    mycursor.close()

In [11]:
auth  = tweepy.OAuthHandler(credentials.API_KEY, credentials.API_SECRET_KEY)
auth.set_access_token(credentials.ACCESS_TOKEN, credentials.ACCESS_TOKEN_SECRET)#watch misspelled "ACCESS_TOEKN"
api = tweepy.API(auth)

In [12]:
myStreamListener = MyStreamListener()
myStream = tweepy.Stream(auth = api.auth, listener = myStreamListener)
myStream.filter(languages=["en"], track = settings.TRACK_WORDS)
# Close the MySQL connection as it finished
# However, this won't be reached as the stream listener won't stop automatically
# Press STOP button to finish the process.
mydb.close()

10 y/os listen to trap music nowadays. I have no words.
Long: None, Lati: None
RT @real_khabri_1: Found something very Apt about #ShehnaazGill

1. Shehnaz Ka Swayamwar task.

2. A favoured call from the caller of the w…
Long: None, Lati: None
RT @sberryeol: exo: works hard, practice over and over again, is amazing and perfect
exo-l: promotes exo, streaming - on all music sites pl…
Long: None, Lati: None
RT @sarareijojo: since we got frozen 2... can we get tangled 2 🥺🥺🥺 https://t.co/t9x16Q78gf
Long: None, Lati: None
RT @TXTranslations: We've got 5 ROTY!!!!!!!!!! 🎊 Congratulations to our 5 amazing hardworking boys!!! 

🏆 - Brand Of The Year
🏆 - M2 X Geni…
Long: None, Lati: None
RT @BLACKPINKFML: 🏆MUSIC OF THE YEAR 🏆

Tags:
Kill This Love  #MúsicaDoAno #POPTimeAwards @ygofficialblink. https://t.co/i22wvDiiLx
Long: None, Lati: None
RT @ujishipjianha: 🏆 13 music show wins
🏆 950k An Ode sales
🏆 Album of the Year
🏆 Group of the Year
🏆AAA Best Social Artist
🏆AAA Best Icon…
Long: None, Lati: No

RT @doyou_bangtan: The 100 Greatest Music Videos of the 2010s: Staff Picks 

69. @BTS_twt’s “Blood, Sweat and Tears” (dir. Choi Yongseek an…
Long: None, Lati: None
RT @lonelyarixo: refreshing twitter every 30 seconds listening to music hbu https://t.co/rrmqfBRGst
Long: None, Lati: None
RT @sza: abs coming in music making sense .. life getting a lil cute ahkay🥺
Long: None, Lati: None
RT @sarareijojo: since we got frozen 2... can we get tangled 2 🥺🥺🥺 https://t.co/t9x16Q78gf
Long: None, Lati: None
RT @chdcofficial: Support👏
Local👏
Music👏
Long: None, Lati: None
I absolutely love Dolly Parton! Watching her 50 years at the Opry. She’s so genuine and so real! And she’s got some… https://t.co/jHq2qqTxTJ
Long: None, Lati: None
RT @arsbum: got7 won awards today that acknowledged their influence in spreading korean culture/kpop music and also acknowledged their powe…
Long: None, Lati: None
RT @Pouyalilpou: new music dec. 13
Long: None, Lati: None
I want to create and produce dope as music and I w

RT @svthingy: [17's] KAMU! IYA KAMU! RT!!!1! LET THE WORLD KNOW SVT IS THAT GOOD 
🏆 13 music show wins
🏆 950k An Ode sales
🏆 Album of the Y…
Long: None, Lati: None
RT @lonelyarixo: refreshing twitter every 30 seconds listening to music hbu https://t.co/rrmqfBRGst
Long: None, Lati: None
why do ya'll get so pressed over an award please that's not healthy for you ma'am go get some fresh air, drink some… https://t.co/7iVcjhxRBs
Long: None, Lati: None
Everyone is like "oh I thought before 2020 we would DEFINITELY have flying cars" but is everyone forgetting when we… https://t.co/9YQUhVbZL4
Long: None, Lati: None
@ftnalsya PLAY IT 🎼🎶🎸🙏 Music lovers Listen to Sweet Baby by Cos Chris on #SoundCloud
https://t.co/zsgtkPrCwc 🔛🔝🔥🔥💎👊🏿💎🔥🔥 SHARE IT
Long: None, Lati: None
RT @CSGO: Today we're excited to ship the @Halo Sticker Capsule in CS:GO! As an added bonus, users who purchase and play The Master Chief C…
Long: None, Lati: None
RT @lonelyarixo: refreshing twitter every 30 seconds listening to mus

RT @jdshawww: *my gf trying to listen to my music in the car with me* 

me: 

her:

me:

her: ok but like what even is a baby birkin
Long: None, Lati: None
RT @pwater41: Watch @XplizitS's broadcast: TUESDAY NITE MIX@djxplizit @periscopeCo #scopelegends #oldschoolhiphop #oldschoolr&amp;b #music #liv…
Long: None, Lati: None
RT @LG_Views: Most viewed Music Videos this year by female artists.

8. Shallow — +573M (747M total) https://t.co/1AC6dVdoZd
Long: None, Lati: None
RT @XOQuestions: Music video on the way!! The Weeknd shuts down The streets of Vegas to film a new video for his upcoming album https://t.c…
Long: None, Lati: None
RT @Raushan57jyg: Happy Birthday to the QUEEN of PUNJABI MUSIC INDUSTRY!😍😍😍
@realhimanshi 

#HappyBirthdayHimanshi https://t.co/loM1ZL3C5T
Long: None, Lati: None
RT @lonelyarixo: refreshing twitter every 30 seconds listening to music hbu https://t.co/rrmqfBRGst
Long: None, Lati: None
RT @lonelyarixo: refreshing twitter every 30 seconds listening to music hbu htt

RT @montanaelliot: Flight attendant: Is there a doctor on this flight?

Dad: *nudging me* that should've been you

Me: Not now Dad

Dad: No…
Long: None, Lati: None
RT @sliccmic: im going to hire someone to follow me around and play clown music everywhere i go
Long: None, Lati: None
RT @mizedub: new music tomorrow 

u aint hear it from me tho
Long: None, Lati: None
RT @BTSVotingFam: [Melon Music Awards]

🏆Artist Award - 1,241,842
🏆Album Award - 1,190,593
🏆Best Song Award - 1,152,289
🏆Netizen Popularity…
Long: None, Lati: None
RT @BTSVotingFam: [Melon Music Awards]

🏆Artist Award - 1,241,842
🏆Album Award - 1,190,593
🏆Best Song Award - 1,152,289
🏆Netizen Popularity…
Long: None, Lati: None
RT @MalluAr48068641: #DecadeForMemorableAarya2 

Awards Received By #Aarya2 
Nominations 

Best Film
Best Director
Best Actor
Best Music Di…
Long: None, Lati: None
LOUDER
Long: None, Lati: None
RT @reidthe3: Abel needs to stop playing games and release his music.
Long: None, Lati: None
RT @MrMouthAlmight

...

RT @purelydowoo: its so funny how people claim “the fandom can’t ruin music for you” because it can. if a fandom is constantly bringing oth…
Long: None, Lati: None
@ITalkFortnite Would you be for STW giving more battle royal rewards, like skins, back blings, and pickaxes, or is… https://t.co/0nRbg5X6q3
Long: None, Lati: None
RT @odetosoonhao: seventeen won a daesang with their self produced album the album most netizens were criticizing for not fitting gp music…
Long: None, Lati: None
RT @lonelyarixo: refreshing twitter every 30 seconds listening to music hbu https://t.co/rrmqfBRGst
Long: None, Lati: None
RT @lavienrougee: 2013 Country Music Awards https://t.co/lhNmlSTrVS
Long: None, Lati: None
the whole tl acting like it's the actual music video being released
Long: None, Lati: None
RT @lavienrougee: 2015 iheart radio music awards https://t.co/bM5LShMtRS
Long: None, Lati: None
Super excited about the new tunes!! ⁦@chadwickstokes⁩

Check out "Chadwick Stokes &amp; The Pintos" by Chadwi

RT @UniversaI: Damn new weeknd dropping. I need to be in a toxic situation to fully enjoy the music
Long: None, Lati: None
RT @machinegunkelly: when im driving around with my daughter listening to music and we’re both rocking out sometimes ill look over at her b…
Long: None, Lati: None
RT @michaeljackson: Grammy Awards, Soul Train Awards, Billboard Awards, American Music Awards, Diamond sales awards…all of these were earne…
Long: None, Lati: None
RT @Ayanimue: 🏆 13 music show wins
🏆 950k An Ode sales
🏆 Album of the Year
🏆 Group of the Year
🏆AAA Best Social Artist
🏆AAA Best Icon
🏆AAA…
Long: None, Lati: None
Am I the only one fucking excited about The Weeknd dropping his new music. Like wtffffff I’m ready ! Lol
Long: None, Lati: None
RT @lonelyarixo: refreshing twitter every 30 seconds listening to music hbu https://t.co/rrmqfBRGst
Long: None, Lati: None
RT @plazaandu: Pink Floyd - Childhood's End (Official Music Video) https://t.co/elgrnGvx0p vía @YouTube
Long: None, Lati: None
RT @lone

RT @intexolteam: 🏴 MUSIC SHOW GUIDE 2019 - EXO🏳️

Keep yourself informed on how to help #EXO win in Music Shows! 🏆

📍Read carefully each cr…
Long: None, Lati: None
RT @odairannies: happy 9th anniversary to tangled, one of the best disney movies out there. just love everything about it: the story, the m…
Long: None, Lati: None
RT @ThulaniDeAfrika: Monique  Bingham

The mother of all house music vocalists a true legend 🙏🙏🙏 https://t.co/2BrxVCkuRc
Long: None, Lati: None
RT @faroit: We are happy to present Open-Unmix - a reference implementation for music source separation.

🔊Demo: https://t.co/mO7nRsq6oQ
📜P…
Long: None, Lati: None
RT @TFIREMUSIC: Sick new song and video from our dudes @saulband !! Stoked that they're part of the HOLIDAY HAVOK TOUR!! Check out all date…
Long: None, Lati: None
RT @itskyleharris: 5 years ago I started Vegas Live Sessions, an intimate night of music, with some of my best friends in Sin City. Now I’m…
Long: None, Lati: None
RT @camilasaenz124: What my manz see

Huuuuuh can y’all just fucking cancel her... her music isn’t even that fucking great for her to be doin y’all like… https://t.co/6MP0W0gzgp
Long: None, Lati: None
RT @sarareijojo: since we got frozen 2... can we get tangled 2 🥺🥺🥺 https://t.co/t9x16Q78gf
Long: None, Lati: None
RT @Carole_King: #ThrowbackBackThursday 
RCA Studios, 1959
Courtesy Sony Music Archives https://t.co/lBCnAVGxV0
Long: None, Lati: None
RT @ArkRazorbacks: Everything really is better with Titanic music.

🚢🐗🏀 https://t.co/VvrVYOEuTV
Long: None, Lati: None
Need dark clouds on Apple Music asap
Long: None, Lati: None
RT @doyou_bangtan: The 100 Greatest Music Videos of the 2010s: Staff Picks 

69. @BTS_twt’s “Blood, Sweat and Tears” (dir. Choi Yongseek an…
Long: None, Lati: None
I'm going to do another music corner
Which group do you think should be in for now?
Long: None, Lati: None
RT @jaboukie: ty for bringing awareness to the fact that no black women has released music during since 2009 https://t.co/1TgzJtTyxd
Long:

KeyboardInterrupt: 